# Explore the Aerosol Optical Depth Metric

In [138]:
from bokeh.io import curdoc, output_notebook
from bokeh.themes import Theme
from bokeh.layouts import layout, row, WidgetBox, Spacer, column
from bokeh.models import ColumnDataSource, LinearColorMapper, LogColorMapper, ColorBar, Line, Panel, Band, Patch
from bokeh.models.widgets import RangeSlider, DateRangeSlider, Dropdown, Toggle
from bokeh.models.formatters import DatetimeTickFormatter
from bokeh.plotting import figure, show
import xarray as xr
import pandas as pd
import numpy as np

## Load in the data

In [117]:
# open the glossac dataset
glossac_file = r'C:\Users\lando\data\netcdf\GloSSAC\GloSSAC-V1.nc'
glossac = xr.open_dataset(glossac_file, decode_times=False)

# read in extinction at 1020nm
extinction = glossac.Measurements_extinction
extinction = extinction.rename({'altitude1': 'altitude',
                                'measurement wavelengths': 'ext_wavel', 
                                'years':'time'})\
                       .sel(ext_wavel=5)
time = pd.date_range('1979', '2016-12-31', freq='MS')
extinction.assign_coords(time=time, latitude=glossac.lat.values, altitude=extinction.altitude.values)

# tile the tropropause over the entire dataset
tropopause = xr.DataArray(np.tile(glossac.trop.values, int(len(glossac.years) / 12)).T,
                          dims=['time', 'latitude'],
                          coords=[time, glossac.lat.values])

## Compute Aerosol Optical Depth

In [122]:
aod = extinction.where(extinction.altitude > tropopause).sum(dim='altitude') * 0.5

global_aod = (np.cos(aod.latitude * np.pi / 180) * aod).sum(dim='latitude') \
             / np.cos(aod.latitude * np.pi / 180).sum(dim='latitude')
background = aod.sel(time=slice('1998', '2002')).groupby('time.month').mean(dim='time')
aod_diff = aod.groupby('time.month') - background
global_aod_diff = (np.cos(aod.latitude * np.pi / 180) * aod_diff).sum(dim='latitude') \
                 / np.cos(aod.latitude * np.pi / 180).sum(dim='latitude')
df = pd.DataFrame(aod.to_pandas().stack(), columns=['Measurements_extinction']).reset_index()
df['width'] = pd.Timedelta(1, 'M')

## Make the Plots

In [49]:
output_notebook()

Loading BokehJS ...

In [157]:
p = figure(plot_height=300, plot_width=900, 
           tooltips=[("time", "$x"), ("latitude", "$y"), ("value", "@Measurements_extinction")],
           x_axis_type='datetime', y_range=(-80, 80))

source = ColumnDataSource(df)
mapper = LinearColorMapper(palette='Viridis256', low=0, high=0.01)
# mapper = LinearColorMapper(palette='Viridis256', low=0.001, high=0.1)

rect = p.rect(x="time", y="latitude", width='width', height=np.diff(aod.latitude)[0],
              source=source,
              fill_color={'field': 'Measurements_extinction', 'transform': mapper},
              line_color={'field': 'Measurements_extinction', 'transform': mapper})
p.yaxis.axis_label = "Latitude"

color_bar = ColorBar(color_mapper=mapper,
                     label_standoff=12, border_line_color=None, location=(0, 0))
p.add_layout(color_bar, 'right')

p2 = figure(plot_height=200, plot_width=900, 
            x_axis_type='datetime', y_axis_type='log', 
            y_range=(1e-4, 1e-1), x_range=p.x_range)
p2.line(global_aod.time.values, global_aod.values, color='gray', legend='AOD')
p2.line(global_aod_diff.time.values, global_aod_diff.values, line_width=2, legend='Deseasonalized')


good = global_aod_diff > 0
upper = global_aod_diff.values[good]
lower =  global_aod_diff.values[good] * 0 + 1e-7
aod_band = ColumnDataSource({'time': np.hstack([global_aod_diff.time.values[good], global_aod_diff.time.values[good][::-1]]),
                             'upper': np.hstack([upper, lower[::-1]])})
# band = Band(base='time', lower='lower', upper='upper', source=aod_band, level='underlay',
#             fill_alpha=1.0, line_width=1, line_color='black')

glyph = Patch(x="time", y="upper", fill_color="#2171b5", line_width=0, fill_alpha=0.3)
p2.add_glyph(aod_band, glyph)

p2.yaxis.axis_label = 'Global AOD [1020nm]'
show(column(p, p2))